<a href="https://colab.research.google.com/github/JoudAlghumaiz/Material_Classifier_TeachableMachine/blob/main/predict_material_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os

# --- Configuration ---
# Path to your Teachable Machine model file (keras_model.h5)
MODEL_PATH = 'keras_model.h5'
# Path to your Teachable Machine labels file (labels.txt)
LABELS_PATH = 'labels.txt'
# Path to the input image you want to classify
# Make sure this image exists in the same directory or provide full path
INPUT_IMAGE_PATH = 'gold.jpg'

# Model's input image dimensions (Teachable Machine usually exports 224x224)
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

# --- Load the Model ---
try:
    model = load_model(MODEL_PATH, compile=False)
    print(f"Model '{MODEL_PATH}' loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'keras_model.h5' is in the same directory or correct path is provided.")
    exit()

# --- Load the Labels ---
class_names = []
try:
    with open(LABELS_PATH, 'r') as f:
        for line in f:
            # Labels usually come with a number prefix (e.g., "0 ClassName")
            # This extracts "ClassName" from "0 ClassName"
            class_names.append(line.strip().split(' ', 1)[1])
    print(f"Labels loaded successfully: {class_names}")
except Exception as e:
    print(f"Error loading labels: {e}")
    print("Please ensure 'labels.txt' is in the same directory or correct path is provided.")
    exit()

# --- Function to preprocess the image ---
def preprocess_image(image_path):
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Error: Input image '{image_path}' not found.")
        return None

    try:
        # Open the image
        image = Image.open(image_path).convert('RGB')

        # Resize the image to be at least (IMAGE_WIDTH, IMAGE_HEIGHT)
        # Teachable Machine's preprocessing often involves padding to square
        # and then resizing. Let's replicate that.
        size = (IMAGE_WIDTH, IMAGE_HEIGHT)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS) # Use LANCZOS for quality

        # Convert image to numpy array
        image_array = np.asarray(image)

        # Normalize the image (Teachable Machine models expect values between -1 and 1)
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Expand dimensions to create a batch of 1 image (required by model.predict)
        data = np.ndarray(shape=(1, IMAGE_WIDTH, IMAGE_HEIGHT, 3), dtype=np.float32)
        data[0] = normalized_image_array

        return data

    except Exception as e:
        print(f"Error processing image '{image_path}': {e}")
        return None

# --- Main prediction logic ---
if __name__ == "__main__":
    print(f"\n--- Starting material classification for '{INPUT_IMAGE_PATH}' ---")

    # Preprocess the input image
    processed_data = preprocess_image(INPUT_IMAGE_PATH)

    if processed_data is not None:
        # Make a prediction
        prediction = model.predict(processed_data)

        # Get the index of the class with the highest probability
        predicted_class_index = np.argmax(prediction)

        # Get the predicted class name
        predicted_class_name = class_names[predicted_class_index]

        # Get the confidence score for the predicted class
        confidence_score = prediction[0][predicted_class_index] * 100

        print(f"\nPrediction for '{INPUT_IMAGE_PATH}':")
        print(f"  Predicted Material Type: {predicted_class_name}")
        print(f"  Confidence: {confidence_score:.2f}%")

        # You can also print all probabilities if desired
        print("\nAll class probabilities:")
        for i, prob in enumerate(prediction[0]):
            print(f"  {class_names[i]}: {prob:.2f}%")
    else:
        print("Image preprocessing failed, cannot make a prediction.")

    print("\n--- Material classification complete ---")

Model 'keras_model.h5' loaded successfully.
Labels loaded successfully: ['Conductors', 'Insulators', 'Semiconductors']

--- Starting material classification for 'gold.jpg' ---
1/1 [==============================] - 1s 1s/step

Prediction for 'gold.jpg':
  Predicted Material Type: Conductors
  Confidence: 98.04%

All class probabilities:
  Conductors: 0.98%
  Insulators: 0.01%
  Semiconductors: 0.01%

--- Material classification complete ---


In [1]:
!pip install tensorflow==2.15.0